In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
!pip install git+https://github.com/tensorflow/docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-m1xhse1c
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-m1xhse1c
  Resolved https://github.com/tensorflow/docs to commit 4d512c2d7c40d69fcb842978aeaa136e19abe2bb
  Preparing metadata (setup.py) ... done


In [73]:
import pickle
import random
import tensorflow as tf
import numpy as np
import nltk
import time
import os

from text_utilities import *
from video_utilities import *
from pipeline_attention_utilities_new import *
from pre_trained_models import *

import nltk
nltk.download('wordnet')
from nltk.translate.meteor_score  import  single_meteor_score as meteor_

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Load Model Weights

In [74]:
embedding_dim = 256
units = 256
vocab_size = 10929
# checkpoint_path_ckpt = "/content/drive/MyDrive/Video_Cap_Data/Checkpoints"
BATCH_SIZE = 1

In [75]:
path = '/content/drive/MyDrive/Video_Cap_Data/Pickles/Tokenizer.pkl'
tokenizer = pickle.load(open(path, 'rb'))      # rb --> read binary
tokenizer.index_word[0]

' '

In [76]:
from keras.models import load_model

encoder_2d = Encoder(embedding_dim)
encoder_3d = Encoder(embedding_dim)
decoder = Decoder(tokenizer, BATCH_SIZE, units, vocab_size, embedding_dim)


encoder_2d.load_weights('/content/drive/MyDrive/Video_Cap_Data/Pickles/encoder_vgg/weights' )  # creates a HDF5 file 'my_model.h5'
encoder_3d.load_weights('/content/drive/MyDrive/Video_Cap_Data/Pickles/encoder_movinet/weights' )  # creates a HDF5 file 'my_model.h5'
decoder.load_weights('/content/drive/MyDrive/Video_Cap_Data/Pickles/decoder/weights')



Uniform entropy: 9.30
Marginal entropy: 5.81


In [77]:
f = open("/content/drive/MyDrive/Video_Cap_Data/Pickles/df_train.pkl", "rb")
df_tst = pickle.load(f)
f.close()

df_tst = df_tst.iloc[:50000 , :]
df_tst.head(3)


,category,video_id,time,caption,split,path
0,2,video2960,12.32,<start> cartoon animals runs through an ice c...,train,/content/drive/MyDrive/Video_Cap_Data/Train_Va...
1,2,video2960,12.32,<start> cartoon character runs around inside ...,train,/content/drive/MyDrive/Video_Cap_Data/Train_Va...
2,2,video2960,12.32,<start> character is running in the snow <end>,train,/content/drive/MyDrive/Video_Cap_Data/Train_Va...


In [78]:
base_path_tst = '/content/drive/MyDrive/Video_Cap_Data/Train_Val_videos/TrainValVideo'

idx = random.randint(0, len(df_tst) )
vid_id = df_tst.iloc[idx , 1]
caption = df_tst.iloc[idx , 3]
vid_ids = [vid_id]

# seqs = get_seqs(tokenizer, caption , 74)

In [79]:
gh_2d =  FrameGenerator(vid_ids, base_path_tst,224, 20)
frames_2d, vid_id = next(gh_2d())

gh_3d =  FrameGenerator(vid_ids, base_path_tst,224, 100 , 2)
frames_3d, vid_id = next(gh_3d())

In [80]:
frames_3d = frames_3d[1:]
frames_2d.shape , frames_3d.shape

((21, 224, 224, 3), (100, 224, 224, 3))

In [81]:
def get_captions(vid_id ):
  df =  df_tst[df_tst['video_id']== vid_id]
  return df.iloc[0 , 3]

In [82]:
get_captions(vid_ids[0])

'<start>  cartoon dog is holding binoculars <end>'

In [83]:
features_2d_path = os.path.join('/content/drive/MyDrive/Video_Cap_Data/features_2d', vid_id[5:]+'.npy')

features_3d_path = os.path.join('/content/drive/MyDrive/Video_Cap_Data/features_3d', vid_id+'.npy')

tensor_2d = tf.constant(np.load(features_2d_path ))
tensor_3d = tf.constant(np.load(features_3d_path ))

tensor_2d.shape , tensor_3d.shape

(TensorShape([21, 4096]), TensorShape([5, 80]))

In [84]:
sz = tensor_2d.shape
if sz[0]!=21:
  diff_2d = 20- sz[0]
  a = tf.zeros((diff_2d, 4096))
  tensor_2d = tf.concat([a, tensor_2d], 0)

sz = tensor_3d.shape
if sz[0]!=5:
  diff_3d = 5- sz[0]
  a = tf.zeros((diff_3d, 480))
  tensor_3d = tf.concat([a, tensor_3d], 0)

tensor_2d = tf.expand_dims(tensor_2d, axis= 0)
tensor_3d = tf.expand_dims(tensor_3d, axis= 0)


tensor_2d.shape , tensor_3d.shape

(TensorShape([1, 21, 4096]), TensorShape([1, 5, 80]))

In [85]:
max_len = 42

dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * BATCH_SIZE, 0)
h = tf.zeros((BATCH_SIZE, units))
c = tf.zeros((BATCH_SIZE, units))

features_2d = encoder_2d(tensor_2d)
features_3d = encoder_3d(tensor_3d)
features_2d.shape , features_3d.shape

(TensorShape([1, 21, 256]), TensorShape([1, 5, 256]))

In [86]:
pred_seqs =None
for i in range(1, max_len):

  pred, h, c = decoder(dec_input, features_2d, features_3d, h ,c)

  dec_input = tf.argmax(pred, axis =-1, output_type=tf.dtypes.int32)
  if tokenizer.index_word[dec_input[0][0].numpy()]=='<end>':
    break
  if pred_seqs == None:
    pred_seqs = dec_input
  pred_seqs = tf.concat([pred_seqs, dec_input], axis= 1)


In [87]:
pred_caption = tokenizer.sequences_to_texts(pred_seqs.numpy())
pred_caption = ' '.join(pred_caption)
real_caption = get_captions(vid_id)

real = real_caption.split(' ')
real_caption = ' '.join(real[1:-1])

In [88]:
print('pred_caption : ' , pred_caption)
print('real_caption : ' , real_caption[1:])
print('Meteor score : ' , meteor_(real_caption.split() , pred_caption.split()))

pred_caption :  man man is talking about the video game
real_caption :  cartoon dog is holding binoculars
Meteor score :  0.09433962264150944
